In [ ]:
%pwd  #look at the current work dir
%cd C:/Users/istoffa/Repos/SECT

%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import datetime

#import umap
import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns
# Import the necessaries libraries
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
#from scipy.cluster.hierarchy import dendrogram, linkage

import utils
import TemporalClusterer
import dCollector as dC
import graphing


import sys
import os

In [ ]:
## Arguments

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
pyo.init_notebook_mode()

dataLoc='Z:\SECT'
tFrom='2020-02-12'
tTo='2020-02-12'
min_events=10
max_activity=0.7
dist_threshold=0.1

In [ ]:
#Load preprocessed files
(df, file_list) = utils.load_files(dataLoc, tFrom, tTo)

tc = TemporalClusterer.TemporalClusterer(min_events=min_events, max_activity=max_activity, dist_threshold=dist_threshold, min_cluster_size=7)

df['labels'] = tc.fit_transform(df, [])

(clusters, series) = tc.post_process(df, file_list)

In [ ]:
print(clusters[['events', 'min_activity', 'min_blocks', 'tfrom', 'tto', 'origins', 'types']])
clusters[['events', 'min_activity', 'min_blocks', 'tfrom', 'tto', 'origins', 'types']].hist()

In [ ]:
clusters[['ips', 'size']]

In [ ]:
sns.heatmap(series.apply(lambda x: np.multiply(x,clusters['size']),axis=0))

In [ ]:
sns.heatmap(series)

In [ ]:
topx=clusters.sort_values(ascending=False, by='size').head(5)
#max=clusters.loc[clusters['size']==clusters['size'].max(),:].head(5)
intervals = utils.sample_intervals(series.loc[topx.index,:],file_list[0],tc.aggregation)

benefizio = dC.dc()
filter = clusters.loc[topx.index,'ips'].apply(dC.dc_liberouter_filter_string).values[0]
res=benefizio.filterFlows(filter, intervals[0][0][0], intervals[0][0][1])
print(res.head())

In [ ]:
fig = go.Figure(graphing.genSankey(res.sample(10),['srcip','dstip'],'packets','Sample of packet flows'))
fig.show()

In [ ]:
badIps = clusters.loc[topx.index,'ips']
print(badIps)
fig = go.Figure(graphing.genSankey(res.loc[res['srcip'].isin(list(badIps.values[0])),:],['srcip','proto'],
                                   'packets', 'Communication by protocols'))
fig.show()

In [ ]:
#reducer = umap.UMAP()
#embedding = reducer.fit_transform(res[['bytes','packets']])
#sns.scatterplot(x=embedding[:][0], y=embedding[:][1])

In [ ]:
sns.lineplot(data=res[['bytes','packets']])

In [ ]:
fig = go.Figure(graphing.genSankey(res.loc[res['srcip'].isin(list(badIps.values[0])),:].sample(20),['srcip','dstip'],'packets','Sample of packet flows'))
fig.show()

In [ ]:
res['bad'] = (res['srcip'].isin(list(badIps.values[0])))
srcdstipflows=res.loc[(res['bad']==True),:].groupby(['srcip', 'dstip'])['packets'].agg('sum')
vc=srcdstipflows.value_counts()
vc.sort_values(inplace=True, ascending=False)
print(vc.head())
srcdstipflows.hist(log=False)

In [ ]:
dstipflows=res.loc[(res['bad']==False),:].groupby([ 'dstip', 'srcip'])['packets'].agg('count')

vc=dstipflows.value_counts()
vc.sort_values(inplace=True, ascending=False)
print(vc.head())
dstipflows.hist(log=False)


In [ ]:
pd.get_dummies(clusters['types'].apply(lambda x: tuple([y for y in [z for z in list(x)]])))


In [ ]:
a=clusters['types'].apply(lambda x: [y.replace('[','').replace(']','').replace(' ','').replace("'",'').replace('"','').split(',') for y in x])#.split(','))#a.value_counts()
a=a.apply(lambda l: tuple(set([item for sublist in l for item in sublist])))
a.apply(print)


In [ ]:
notScans=clusters.loc[(a!=('Recon.Scanning',)),['events', 'min_activity', 'min_blocks', 'tfrom', 'tto', 'origins', 'types']]
notScans.apply(print, axis=1)
notScans[['events', 'min_activity', 'min_blocks', 'tfrom', 'tto', 'origins', 'types']].hist()

In [ ]:
res.describe()


In [ ]:
(res['first'].max()-res['first'].min())/60000